## - Import

In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import os 
from torchvision.datasets import SVHN 
from torchvision.transforms import ToTensor

from tqdm import tqdm

import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchvision.utils import save_image#
import shutil

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [ ]:
torch.cuda.empty_cache()

In [166]:
from torch.utils.tensorboard import SummaryWriter

GAN_LOGS = os.path.join(os.getcwd(), "tboard_logs", "gan")
if  os.path.exists(GAN_LOGS):
    shutil.rmtree(GAN_LOGS)
if not os.path.exists(GAN_LOGS):
    os.makedirs(GAN_LOGS)

writer = SummaryWriter(GAN_LOGS)

In [165]:
if  os.path.exists("imgs/training"):
    shutil.rmtree("imgs/training")

if not os.path.exists("imgs/training"):
    os.makedirs("imgs/training")

## - Parameters

In [ ]:
transform = transforms.Compose([
        # transforms.Grayscale(),
        # transforms.Resize(128),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5,), std=(0.5,))
])

In [ ]:
train_dataset=SVHN(split='train',root='data/train',transform=transform,download=False)
# test_dataset=SVHN(split='test',root='data/test',transform=transform)

# vars(train_dataset)

In [ ]:
class_list = {'0','1','2','3','4','5','6','7','8','9'}

In [ ]:
# Data


img_size = 32 # Image size
batch_size = 64  # Batch size

# Model
z_size = 500
generator_layer_size = [128, 512, 1024]
discriminator_layer_size = [1024, 512, 128]

# Training
epochs = 50  # Train epochs
learning_rate = 1e-4
class_num=10
channels=3

In [ ]:
train_loader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=8, pin_memory=True)
# val_loader = DataLoader(test_dataset, batch_size*2, num_workers=4, pin_memory=True)

In [ ]:
a,b=next(iter(train_loader))
a.shape

## - Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, generator_layer_size, z_size, img_size, class_num):
        super().__init__()
        
        self.z_size = z_size
        self.img_size = img_size
        
        self.label_emb = nn.Embedding(class_num, class_num)
        
        self.model = nn.Sequential(
            nn.Linear(self.z_size + class_num, generator_layer_size[0]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(generator_layer_size[0], generator_layer_size[1]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(generator_layer_size[1], generator_layer_size[2]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(generator_layer_size[2], self.img_size * self.img_size*channels),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        
        # Reshape z
        z = z.view(-1, self.z_size)
        
        # One-hot vector to embedding vector
        c = self.label_emb(labels)
        
        # Concat image & label
        x = torch.cat([z, c], 1)
        # Generator out
        out = self.model(x)
        
        return out.view(-1,channels, self.img_size, self.img_size)

## - Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, discriminator_layer_size, img_size, class_num):
        super().__init__()
        
        self.label_emb = nn.Embedding(class_num, class_num)
        self.img_size = img_size
        
        self.model = nn.Sequential(
            nn.Linear(self.img_size * self.img_size*channels + class_num, discriminator_layer_size[0]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(discriminator_layer_size[0], discriminator_layer_size[1]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(discriminator_layer_size[1], discriminator_layer_size[2]),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(discriminator_layer_size[2], 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
        
        # Reshape fake image
        x = x.view(-1, channels*self.img_size * self.img_size)
        # One-hot vector to embedding vector
        c = self.label_emb(labels)
        
        # Concat image & label
        x = torch.cat([x, c], 1)
        
        # Discriminator out
        out = self.model(x)
        # return out
        return out.squeeze()

In [ ]:
# Define generator
generator = Generator(generator_layer_size, z_size, img_size, class_num).to(device)
# Define discriminator
discriminator = Discriminator(discriminator_layer_size, img_size, class_num).to(device)

## - Adversarial Learning of Generator & Discriminator

In [ ]:
# Loss function
criterion = nn.BCELoss()

In [ ]:
# Optimizer
g_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)

In [ ]:
criterion_d_real = lambda pred: F.binary_cross_entropy(pred, torch.ones(pred.shape[0], device=pred.device))
criterion_d_fake = lambda pred: F.binary_cross_entropy(pred, torch.zeros(pred.shape[0], device=pred.device))
criterion_g = lambda pred: F.binary_cross_entropy(pred, torch.ones(pred.shape[0], device=pred.device))

In [ ]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    
    # Init gradient
    g_optimizer.zero_grad()
    
    # Building z
    z = Variable(torch.randn(batch_size, z_size)).to(device)
    
    # Building fake labels
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, class_num, batch_size))).to(device)
    
    # Generating fake images
    fake_images = generator(z, fake_labels)
    
    # Disciminating fake images
    validity = discriminator(fake_images, fake_labels)
    # print(validity.shape)
    # Calculating discrimination loss (fake images)
    # g_loss = criterion(validity, Variable(torch.ones(batch_size)).to(device))
    # g_loss = criterion(validity, torch.ones(validity.shape[0], device=validity.device))
    g_loss = criterion_g(validity.view(batch_size))
    # Backword propagation
    g_loss.backward()
    
    #  Optimizing generator
    g_optimizer.step()
    
    return g_loss.data

In [ ]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    
    # Init gradient 
    d_optimizer.zero_grad()
    B = real_images.shape[0]

    # Disciminating real images
    real_validity = discriminator(real_images, labels)
    
    # Calculating discrimination loss (real images)
    # real_loss = criterion(real_validity, Variable(torch.ones(batch_size)).to(device))
    # real_loss = criterion(real_validity, torch.ones(real_validity.shape[0], device=real_validity.device))
    real_loss =criterion_d_real(real_validity)
    
    # Building z
    z = Variable(torch.randn(batch_size, z_size)).to(device)
    
    # Building fake labels
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, class_num, batch_size))).to(device)
    
    # Generating fake images
    fake_images = generator(z, fake_labels)
    
    # Disciminating fake images
    fake_validity = discriminator(fake_images, fake_labels)
    # print(fake_validity.shape)
    
    # Calculating discrimination loss (fake images)
    # fake_loss = criterion(fake_validity, Variable(torch.zeros(batch_size)).to(device))
    # fake_loss = criterion(fake_validity, torch.ones(fake_validity.shape[0], device=fake_validity.device))
    fake_loss=criterion_d_fake(fake_validity.view(B))
    # Sum two losses
    d_loss = real_loss , fake_loss
    (real_loss + fake_loss).backward()

    
    # Optimizing discriminator
    d_optimizer.step()
    
    return d_loss

In [ ]:
def train_gan(epochs,discriminator,train_loader,generator,g_optimizer=g_optimizer,batch_size=batch_size,d_optimizer=d_optimizer):
    iter_ = 0
    for epoch in range(epochs):
        
        print('Starting epoch {}...'.format(epoch+1))
        
        for i, (images, labels) in enumerate(train_loader):
            
            # Train data
            real_images = Variable(images).to(device)
            labels = Variable(labels).to(device)
            
            # Set generator train
            generator.train()
            
            # Train discriminator
            d_loss_real, d_loss_fake = discriminator_train_step(len(real_images), discriminator,
                                            generator, d_optimizer, criterion,
                                            real_images, labels)
            d_loss = d_loss_real + d_loss_fake
            
            # Train generator
            g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)
            writer.add_scalar(f'Loss/Generator Loss', g_loss.item(), global_step=iter_)
            writer.add_scalar(f'Loss/Discriminator Loss', d_loss_real.item(), global_step=iter_)
            writer.add_scalars(f'Loss/Discriminator Losses', {
                        "Real Images Loss": d_loss_real.item(),
                        "Fake Images Loss": d_loss_fake.item(),
                    }, global_step=iter_)
            writer.add_scalars(f'Comb_Loss/Losses', {
                            'Discriminator': d_loss.item(),
                            'Generator':  g_loss.item()
                        }, iter_) 
            iter_=iter_+1
        
        # Set generator eval
        # generator.eval()
        
            # print('g_loss: {}, d_loss: {}'.format(g_loss, d_loss))
        
        # Building z 

            # print(sample_images.shape)
            # Show images
            if(iter_ % 2000 == 0):
                with torch.no_grad():
                    generator.eval()
                    z__ = Variable(torch.randn(class_num-1, z_size)).to(device)
                
                # Labels 0 ~ 9
                    labels__ = Variable(torch.LongTensor(np.arange(class_num-1))).to(device)
                    plt.figure(figsize=(6,6))
                
                # Generating images
                    sample_images = generator(z__, labels__).data.cpu()
                    grid = make_grid(sample_images, nrow=4, normalize=True)
                    # .permute(1,2,0).numpy()
                    writer.add_image('images', grid, global_step=iter_)
                    torchvision.utils.save_image(grid, os.path.join(os.getcwd(), "imgs", "training", f"imgs_{iter_}.png"))
                  # 
                    plt.imshow(grid.permute(1,2,0).numpy())
                    plt.show()

In [ ]:
train_gan(epochs=epochs,generator=generator,discriminator=discriminator,train_loader=train_loader)

## - Show Generating Images

In [ ]:
z2 = Variable(torch.randn(z_size, 9)).to(device)

# Labels 0 ~ 9
tt=[i for i in range(9)]
labels2 = Variable(torch.LongTensor([i for i in  tt]) ).to(device)

sample_images = generator(z2, labels2).data.cpu()

# Show images
grid = make_grid(sample_images, nrow=3, normalize=True, scale_each=True).permute(1,2,0).numpy()
fig, ax = plt.subplots(figsize=(6,6))
ax.imshow(grid)
_ = plt.yticks([])
# _ = plt.xticks(np.arange(15, 300, 30), class_list, rotation=45, fontsize=20)